In [1]:
import java.util.ArrayList;
import java.util.List;

public class Horse {
    private String name;
    private char symbol;
    private char originalSymbol;  
    private double confidence;
    private String breed;
    private String color;
    private String saddle;
    private String horseshoes;
    private String accessory;
    private int position;
    private int racesParticipated;
    private int racesWon;
    private List<Double> raceTimes;
    private boolean fallen;      
    private double fallRisk;     

    public Horse(String name, char symbol, double confidence, String breed, String color, String saddle, String horseshoes, String accessory) {
        this.name = name;
        this.symbol = symbol;
        this.originalSymbol = symbol; 
        this.confidence = confidence;
        this.breed = breed;
        this.color = color;
        this.saddle = saddle;
        this.horseshoes = horseshoes;
        this.accessory = accessory;
        this.position = 0;
        this.racesParticipated = 0;
        this.racesWon = 0;
        this.raceTimes = new ArrayList<>();
        this.fallen = false;
        
        this.fallRisk = 0.01 + Math.random() * 0.04;
        if (horseshoes.toLowerCase().contains("cheap") || horseshoes.toLowerCase().contains("old")) {
            this.fallRisk += 0.03;  
        }
    }

    public void move() {
        if (fallen) {
            return;
        }
        if (Math.random() < fallRisk) {
            fall();
            return;
        }
        
        double moveChance = Math.random();
        if (moveChance < confidence) {
            position += 1;
        }
    }
    
    public void fall() {
        fallen = true;
        symbol = '❌'; 
        System.out.println("\n💥 " + name + " has fallen!");
    }
    
    public boolean hasFallen() {
        return fallen;
    }
    
    public void resetPosition() {
        position = 0;
        fallen = false;
        symbol = originalSymbol;  
    }

    public void recordRace(double timeTaken, boolean won) {
        racesParticipated++;
        if (won) racesWon++;
        raceTimes.add(timeTaken);
    }

    public double getAverageSpeed(int trackLength) {
        if (raceTimes.isEmpty()) return 0;
        double totalSpeed = 0;
        for (double time : raceTimes) {
            totalSpeed += (trackLength / time);
        }
        return totalSpeed / raceTimes.size();
    }

    public double getWinRatio() {
        if (racesParticipated == 0) return 0;
        return (double) racesWon / racesParticipated * 100;
    }

    public List<Double> getRaceTimes() {
        return raceTimes;
    }

    public int getPosition() {
        return position;
    }

    public String getName() {
        return name;
    }

    public char getSymbol() {
        return symbol;
    }
    
    public double getConfidence() {
        return confidence;
    }
}


In [2]:
public class StatisticsManager {
    public void displayHorseStatistics(Horse horse, int trackLength) {
        System.out.println("\n--- Horse Statistics for " + horse.getName() + " ---");
        System.out.println("Total Races: " + horse.getRaceTimes().size());
        System.out.println("Win Ratio: " + String.format("%.2f", horse.getWinRatio()) + "%");
        System.out.println("Average Speed: " + String.format("%.2f", horse.getAverageSpeed(trackLength)) + " units per second");
        System.out.println("Race Times: " + horse.getRaceTimes());
    }

    public void compareHorses(Horse horse1, Horse horse2, int trackLength) {
        System.out.println("\n--- Horse Comparison ---");
        System.out.println(horse1.getName() + " - Avg Speed: " + String.format("%.2f", horse1.getAverageSpeed(trackLength)) + " Win Ratio: " + String.format("%.2f", horse1.getWinRatio()) + "%");
        System.out.println(horse2.getName() + " - Avg Speed: " + String.format("%.2f", horse2.getAverageSpeed(trackLength)) + " Win Ratio: " + String.format("%.2f", horse2.getWinRatio()) + "%");
    }
}


In [3]:
import java.util.HashMap;
import java.util.Map;
import java.util.Random;

public class BettingSystem {
    private double balance;
    private Map<Horse, Double> odds;
    private Map<Horse, Double> bets;

    public BettingSystem() {
        this.balance = 1000.0; 
        this.odds = new HashMap<>();
        this.bets = new HashMap<>();
    }

    public void calculateOdds(Horse[] horses) {
        Random rand = new Random();
        odds.clear();
        for (Horse horse : horses) {
            if (horse != null) {
                double base = 1.5 + rand.nextDouble() * 2.5; 
                odds.put(horse, base);
            }
        }
    }

    public void displayOdds() {
        System.out.println("\n--- Betting Odds ---");
        for (Horse horse : odds.keySet()) {
            System.out.println(horse.getName() + " (" + horse.getSymbol() + "): " + String.format("%.2f", odds.get(horse)) + "x");
        }
    }

    public void placeBet(Horse horse, double amount) {
        if (amount > balance) {
            System.out.println("Not enough balance to place bet!");
            return;
        }
        balance -= amount;
        bets.put(horse, amount);
    }

    public void processRaceResult(Horse winner) {
        if (bets.containsKey(winner)) {
            double winnings = bets.get(winner) * odds.get(winner);
            System.out.println("You won! You earned: $" + String.format("%.2f", winnings));
            balance += winnings;
        } else {
            System.out.println("You lost your bet.");
        }
        bets.clear();
    }
    
    public void processNoWinner() {
        System.out.println("Since no horse finished the race, all bets are lost.");
        bets.clear();
    }

    public void displayBalance() {
        System.out.println("Current Balance: $" + String.format("%.2f", balance));
    }
}


In [4]:
public class Race {
    private Horse[] horses;
    private int trackLength;
    private int numLanes;
    private StatisticsManager statsManager;
    private BettingSystem bettingSystem;

    public Race(int trackLength, int numLanes) {
        this.trackLength = trackLength;
        this.numLanes = numLanes;
        horses = new Horse[numLanes];
        statsManager = new StatisticsManager();
        bettingSystem = new BettingSystem();
    }

    public void addHorse(Horse horse, int lane) {
        if (lane >= 0 && lane < horses.length) {
            horses[lane] = horse;
        }
    }

    public int getNumLanes() {
        return numLanes;
    }

    public int getNumHorses() {
        int count = 0;
        for (Horse horse : horses) {
            if (horse != null) count++;
        }
        return count;
    }

    public Horse[] getHorses() {
        return horses;
    }

    public BettingSystem getBettingSystem() {
        return bettingSystem;
    }

    public void runRace() {
        boolean raceFinished = false;
        long startTime = System.currentTimeMillis();
        System.out.println("\n🏁 The race begins!");
        
        while (!raceFinished) {
            for (Horse horse : horses) {
                if (horse != null) {
                    horse.move();
                    if (horse.getPosition() >= trackLength) {
                        raceFinished = true;
                    }
                }
            }
            
            printTrack();
            
            if (allHorsesFallenOrFinished()) {
                raceFinished = true;
            }
            
            try {
                Thread.sleep(500);
            } catch (InterruptedException e) {
                e.printStackTrace();
            }
        }
        
        long endTime = System.currentTimeMillis();
        double timeTaken = (endTime - startTime) / 1000.0; // seconds
        
        Horse winner = null;
        for (Horse horse : horses) {
            if (horse != null && horse.getPosition() >= trackLength) {
                winner = horse;
                System.out.println("\n🏆 " + horse.getName() + " wins the race!");
                break;
            }
        }
        
        if (winner == null) {
            System.out.println("\n😱 No horse finished the race! All horses have fallen!");
        }
        
        for (Horse horse : horses) {
            if (horse != null) {
                horse.recordRace(timeTaken, horse == winner);
                horse.resetPosition();
            }
        }
        
        if (winner != null) {
            bettingSystem.processRaceResult(winner);
        } else {
            System.out.println("All bets are lost as no horse finished the race.");
            bettingSystem.processNoWinner();
        }
    }
    
    private void printTrack() {
        StringBuilder track = new StringBuilder();
        for (int i = 0; i < numLanes; i++) {
            Horse horse = horses[i];
            if (horse != null) {
                track.append(horse.getName())
                     .append(" - Position: ").append(horse.getPosition())
                     .append(" Symbol: ").append(horse.getSymbol())
                     .append("\n");
            }
        }
        System.out.println(track.toString());
    }

    private boolean allHorsesFallenOrFinished() {
        for (Horse horse : horses) {
            if (horse != null && !horse.hasFallen() && horse.getPosition() < trackLength) {
                return false;
            }
        }
        return true;
    }
}


In [5]:
import javax.swing.*;
import java.awt.*;
import java.util.ArrayList;

public class RaceGUI extends JPanel {
    private ArrayList<Horse> horses;
    private int trackLength;
    
    public RaceGUI(ArrayList<Horse> horses, int trackLength) {
        this.horses = horses;
        this.trackLength = trackLength;
        setPreferredSize(new Dimension(800, 400));  // Set the panel size
    }
    
    @Override
    protected void paintComponent(Graphics g) {
        super.paintComponent(g);
        drawTrack(g);
        drawHorses(g);
    }
    
    private void drawTrack(Graphics g) {
        g.setColor(Color.GRAY);
        g.fillRect(0, 50, trackLength * 10, 30); // Drawing the race track
    }

    private void drawHorses(Graphics g) {
        for (Horse horse : horses) {
            g.setColor(Color.BLACK);
            g.fillOval(horse.getPosition() * 10, 50, 20, 20); // Drawing horses as circles
        }
    }

    public void updateHorsePositions() {
        for (Horse horse : horses) {
            horse.move();
        }
        repaint();
    }

    public void startRace() {
        while (true) {
            updateHorsePositions();
            try {
                Thread.sleep(500); // Pause to simulate time passing in the race
            } catch (InterruptedException e) {
                e.printStackTrace();
            }
        }
    }
}


In [9]:
import java.util.ArrayList;
import java.util.List;
import java.util.Scanner;

public class Main {

    public static void main(String[] args) {
        Scanner scanner = new Scanner(System.in);
        Race race = null;
        RaceGUI raceGUI = null;

        while (true) {
            System.out.println("\n=== Horse Race Simulator ===");
            System.out.println("1. Create Track");
            System.out.println("2. Create Horses");
            System.out.println("3. Place Bets");
            System.out.println("4. Start Race");
            System.out.println("5. View Statistics");
            System.out.println("6. Start Graphical Race");
            System.out.println("7. Exit");
            System.out.print("Choose an option: ");
            int choice = scanner.nextInt();
            scanner.nextLine();

            if (choice == 1) {
                race = createTrack(scanner);
            } else if (choice == 2) {
                if (race == null) {
                    System.out.println("You must create a track first!");
                } else {
                    createHorses(scanner, race);
                }
            } else if (choice == 3) {
                if (race != null) {
                    placeBets(scanner, race);
                } else {
                    System.out.println("Create a track and horses first!");
                }
            } else if (choice == 4) {
                if (race != null) {
                    race.runRace();
                } else {
                    System.out.println("Create a track and horses first!");
                }
            } else if (choice == 5) {
                if (race != null) {
                    race.displayHorseStatistics();
                } else {
                    System.out.println("No race data yet!");
                }
            } else if (choice == 6) {
                // Convert Horse[] to ArrayList<Horse>
                ArrayList<Horse> horseList = new ArrayList<>();
                for (Horse horse : race.getHorses()) {
                    if (horse != null) {
                        horseList.add(horse);
                    }
                }

                // Start the graphical race
                raceGUI = new RaceGUI(horseList, 100); // Example track length
                raceGUI.startRaceGUI();
            } else if (choice == 7) {
                System.out.println("Goodbye!");
                break;
            } else {
                System.out.println("Invalid choice. Try again!");
            }
        }
        scanner.close();
    }

    private static Race createTrack(Scanner scanner) {
        System.out.print("Enter track length: ");
        int length = scanner.nextInt();
        System.out.print("Enter number of lanes: ");
        int lanes = scanner.nextInt();
        scanner.nextLine();
        return new Race(length, lanes);
    }

    private static void createHorses(Scanner scanner, Race race) {
        for (int i = 0; i < race.getNumLanes(); i++) {
            System.out.println("\nCreating Horse for Lane " + (i + 1));

            System.out.print("Enter horse name: ");
            String name = scanner.nextLine();

            System.out.print("Enter horse breed: ");
            String breed = scanner.nextLine();

            System.out.print("Enter coat color: ");
            String color = scanner.nextLine();

            System.out.print("Enter horse symbol (e.g. 🐎): ");
            char symbol = scanner.nextLine().charAt(0);

            System.out.print("Choose saddle: ");
            String saddle = scanner.nextLine();

            System.out.print("Choose horseshoes (quality affects fall risk): ");
            String horseshoes = scanner.nextLine();

            System.out.print("Choose an accessory: ");
            String accessory = scanner.nextLine();

            double confidence = 0.7 + Math.random() * 0.3;

            Horse horse = new Horse(name, symbol, confidence, breed, color, saddle, horseshoes, accessory);
            race.addHorse(horse, i);
        }
    }

    private static void placeBets(Scanner scanner, Race race) {
        BettingSystem bettingSystem = race.getBettingSystem();
        bettingSystem.calculateOdds(race.getHorses());
        bettingSystem.displayOdds();

        System.out.print("Enter lane number to bet on (1-" + race.getNumLanes() + "): ");
        int lane = scanner.nextInt() - 1;
        System.out.print("Enter bet amount: ");
        double amount = scanner.nextDouble();
        scanner.nextLine();

        if (lane >= 0 && lane < race.getHorses().length) {
            Horse horse = race.getHorses()[lane];
            if (horse != null) {
                bettingSystem.placeBet(horse, amount);
            } else {
                System.out.println("No horse in that lane!");
            }
        } else {
            System.out.println("Invalid lane number!");
        }

        bettingSystem.displayBalance();
    }
}
